## Google Cloud API

In [1]:
import io
import os
import csv
from time import sleep
from random import *
from file_util import *

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

# Instantiates a client
client = speech.SpeechClient()

In [2]:
# The name of the audio file to transcribe
file_name = '../data/raw/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro02/Ses01F_impro02_M011.wav'

#### IEMOCAP

In [54]:
# list_files = []

# for x in xrange(5):
#     sess_name = 'Session' + str(x+1)

#     path = '../data/raw/IEMOCAP_full_release/' + sess_name + '/sentences/wav/'
#     file_search(path, list_files)
#     list_files = sorted(list_files)

#     print sess_name + ", #sum files: " + str(len(list_files))

#### MSP-IMPROV

In [3]:
list_files = []

path = '../data/raw/MSP_IMPROV/Audio/'
file_search(path, list_files)
list_files = sorted(list_files)
len(list_files)

8438

## RUN

In [4]:
def get_trans( target_files, result_file_name ) :

    cnt = 0

    for file_name in target_files:

        # Loads the audio into memory
        with io.open(file_name, 'rb') as audio_file:
            content = audio_file.read()
            audio = types.RecognitionAudio(content=content)

        config = types.RecognitionConfig(
#             encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
#             sample_rate_hertz=16000,
            
#             encoding=enums.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
#             sample_rate_hertz=48000,
            language_code='en-US')

        # Detects speech in the audio file
        response = client.recognize(config, audio)


        ff = open(result_file_name, 'a')
        writer = csv.writer(ff)

        ret_trans = None
        ret_confidence = None
        
        for result in response.results:
            ret_trans      = result.alternatives[0].transcript
            ret_confidence = result.alternatives[0].confidence
#             print 'Transcript: {}'.format(ret_trans)
#             print 'Transcript: {}'.format(ret_confidence)
        
        writer.writerow([file_name.split('/')[-1][:-4], ret_trans, ret_confidence])    
        cnt += 1
        sleep( (random()/10) )
        
        if (cnt % 500 == 0) :
            print cnt

        ff.close()    

In [61]:
# sample = []
# sample.append(list_files[2])
# print list_files[2]

# get_trans( sample, '../data/raw/MSP_G_SPEECH.csv')

In [108]:
print len(list_files)
get_trans( list_files[6731:], '../data/raw/MSP_G_SPEECH.csv')

8438
500
1000
1500


In [107]:
list_files[6730]

'../data/raw/MSP_IMPROV/Audio/session5/S15A/S/MSP-IMPROV-S15A-F05-S-FM01.wav'

#### for missing files

In [34]:
missing_full_path = []

for x, id in enumerate(target_files) :
    if id.split('/')[-1][:-4] in missing_id:
        missing_full_path.append(  id  )

In [110]:
missing_full_path[0]

'../data/raw/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro04/Ses05M_impro04_F009.wav'

In [111]:
get_trans( missing_full_path, '../data/raw/IEMOCAP_G_SPEECH_missing.csv')

In [109]:
# The name of the audio file to transcribe
missing_full_path = []
missing_full_path.append( '../data/raw/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro04/Ses05M_impro04_F009.wav' )
# missing_full_path.append( '../data/raw/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro02/Ses01F_impro02_F000.wav' )

### IEMOCAP merge multi-processed transcription

In [50]:
speech_ = []
speech_id = []
speech_trans = []
with open( '../data/raw/IEMOCAP_G_SPEECH.csv') as f:
    reader = csv.reader(f)
    speech_ = [ [x[0], x[1]] for x in reader ]        

speech_id = [ x[0] for x in speech_ ]
speech_trans = [ x[1] for x in speech_ ]

In [51]:
with open('../data/raw/IEMOCAP_G_SPEECH_merge.csv', 'w') as f:

    writer = csv.writer(f)

    sent = ''
    for i, id in enumerate(speech_id):

        # last case
        if i == len(speech_id)-1 :
            sent = sent + speech_trans[i]
            writer.writerow( [speech_id[i], sent] )
            break
            
        if speech_id[i] == speech_id[i+1] :
            sent = sent + speech_trans[i]
        
        else :
            sent = sent + speech_trans[i]
            writer.writerow( [speech_id[i], sent] )
            sent = ''

In [52]:
missing_id = []

In [59]:
tmp_id = []
with open('../data/processed/IEMOCAP/processed_ids.txt') as f:
    tmp_id = f.readlines()
tmp_id = [ x.strip() for x in tmp_id]    

In [60]:
for id in tmp_id:
    if id not in speech_id:
        missing_id.append(id)

In [61]:
len(missing_id)

529

In [ ]:
../data/raw/IEMOCAP_full_release/Session5/sentences/wav/Ses05M_impro04/Ses05M_impro04_F009.wav

## merge trans

In [119]:
speech_ = []
with open( '../data/raw/IEMOCAP_G_SPEECH_merge.csv') as f:
    reader = csv.reader(f)
    speech_ = [ [x[0], x[1]] for x in reader ]        
len(speech_)

5002

In [120]:
trans = []
with open('../data/processed/IEMOCAP/processed_ids.txt') as f:
    reader = csv.reader(f)
    trans = [ x for x in reader ]
    
for id, text in trans:
    if id in missing_id:
        speech_.append( [id, text] )
        
speech_ = sorted(speech_)

len(speech_)            

5531

In [107]:
with open('../data/processed/IEMOCAP/processed_tran_fromG.csv', 'w')  as f:
    writer = csv.writer(f)
    for id, text in speech_:
        writer.writerow( [id, text] )

In [132]:
# validate 

list1 = []
list2 =[]
with open('../data/processed/IEMOCAP/processed_ids.txt') as f:
    list1 = f.readlines()
list1 = [x.strip() for x in list1]    
    
with open('../data/processed/IEMOCAP/processed_tran_fromG.csv') as f:
    reader = csv.reader(f)
    list2 = [ x[0] for x in reader ]    

In [133]:
for x in xrange( len(list1 )) :
    if list1[x] != list2[x]:
        print list1[x], list2[x]

## merge - MSP G and MS

In [5]:
g_trans = []
with open('../data/raw/MSP_G_SPEECH.csv') as f:
    reader = csv.reader(f)
    g_trans = [ x for x in reader ]
len(g_trans)    

8438

In [6]:
ms_files = []
ms_trans = []
file_search('../data/raw/MSP_IMPROV/MSP-IMPROV_text/', ms_files)
len(ms_files)

for x in ms_files:
    with open(x) as f:
        ms_trans.append( [ x.split('/')[-1][:-4], f.readlines()[0].strip() ]  )

ms_ids = [ x[0].strip() for x in ms_trans ]

In [7]:
new_g_trans = []
for x in g_trans:
    if x[1] != '':
        new_g_trans.append([x[0], x[1], x[2]])
    else:
        find_data = '_NA_'
        for data in ms_trans:
            if x[0] == data[0]:
                find_data = data[1]
        x[1] = find_data
        x[2] = 0.000
        new_g_trans.append([x[0], x[1], x[2]])

new_g_trans = sorted(new_g_trans)

In [8]:
with open('../data/raw/MSP_G_SPEECH_MS_merge.csv', 'w') as f:
    writer = csv.writer(f)
    for x in new_g_trans:
        writer.writerow( [x[0], x[1], x[2] ])        

In [10]:
with open('../data/processed/MSP_IMPROV/id_from_trans.txt', 'w') as f:
    for x in new_g_trans:
        f.write( x[0] + '\n')

In [205]:
miss_G = []
miss_G = [ x for x in g_trains if x[1] == '']
len(miss_G)

640

In [206]:
new_miss_G = []
new_miss_G = [ x for x in new_g_trans if x[1] == '_NA_']
len(new_miss_G)

453